# Step 3: Using your Graph

In step 3 of this tutorial, we use our cleaned graph to create an Origin-Destination matrix (OD). Our setting remains Reykjavik, Iceland, as we look at travel times along the network to churches.

In [ ]:
# This is a Jupyter Notebook extension which reloads all of the modules whenever you run the code
# This is optional but good if you are modifying and testing source code
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import networkx as nx
import geopandas as gpd
import pandas as pd
import pickle as pkl

In [ ]:
# import GOSTnet library
import GOSTnets as gn

In [ ]:
from shapely.geometry import Point

First, we read in the graph from the result of the cleaning process (Step 2)

In [ ]:
pth = "./"  # change this path to your working folder
G = pkl.load(
    open(os.path.join(pth, "tutorial_outputs", r"iceland_network_clean.pickle"), "rb")
)

At this stage each edge in the network has a property called 'length'. This was actually computed during Step 1 when the generateRoadsGDF function was run. The units of this length are in kilometres.

In [ ]:
gn.example_edge(G)

We want to convert length to time, so that we can conduct analysis on how long it takes to reach certain destinations.  

We do this using the convert_network_to_time function. We have used a factor of 1000, because the function is expecting meters, so we need to convert the units of kilometers to meters.   

The convert_network_to_time function uses a default speed dictionary that assigns speed limits to OSM highway types. However, it is possible to specify your own speed dictionary.

In [ ]:
G_time = gn.convert_network_to_time(
    G, distance_tag="length", road_col="infra_type", factor=1000
)

We can now use the 'time' property for each edge to work out how long it takes to get from one node to another!

In [ ]:
gn.example_edge(G_time, 1)

To do this for just one journey, we could call nx.shortest_path_length on any given origin or destination node. Let's list 10 of our nodes using this networkX function:

In [ ]:
list(G_time.nodes)[:10]

In [ ]:
A = list(G_time.nodes)[0]  # first node in list
B = list(G_time.nodes)[10]  # 10th node in list
travel_time = nx.shortest_path_length(G_time, A, B, weight="time")
print(
    "The travel time between A and B is: %d seconds, or %d minutes!"
    % (travel_time, travel_time / 60)
)

In our example, we want to use our network for Reykjavik to work out the travel time to local churches.

Here, we import a shapefile for Reykjavik, and reproject it to WGS 84:

In [ ]:
rek = gpd.read_file(os.path.join(pth, "tutorial_data", "rek2.shp"))
rek = rek.to_crs("epsg:4326")

Next, We set a variable poly equal to just the geometry

In [ ]:
poly = rek.geometry.iloc[0]

We can visualize this in-line by just calling it:

In [ ]:
poly

With this in hand, we can read in a shapefile of destinations - here, the churches in Iceland. We use Shapely's 'within' command to select just those in the Reykjavik area:

In [ ]:
churches = gpd.read_file(os.path.join(pth, "tutorial_data", "churches.shp"))
churches = churches.loc[churches.within(poly)]

In order to perform network analysis we want to know the closest network node to each church. For this, we use the pandana snap function to snap the church locations to the road network:

In [ ]:
churches

In [ ]:
# the crs of churches
churches.crs

In [ ]:
# view the pandana_snap doc string
gn.pandana_snap?

We want the nearest node distance (NN_dist) to be measured in meters, so that is why we include the target_crs parameter specifying the correct UTM zone.

In [ ]:
churches = gn.pandana_snap_c(
    G_time,
    churches,
    source_crs="epsg:4326",
    target_crs="epsg:32627",
    add_dist_to_node_col=True,
)

As we can see from the NN_dist column, our church locations are very close to a node on the network in all cases

In [ ]:
churches

When calculating an OD-Matrix, we can only use the node IDs as inputs. So, we convert this column of our dataframe over to a list of unique values:

In [ ]:
destinations = list(set(churches.NN))

In [ ]:
destinations

## Further Analysis

We would like to make an OD matrix where the origin is the cottage we are renting in the city, and the destinations are the churches in Reykjavik. This will help us work out how many churches we can see today!. First, we need to create the origin. It has coordinates: 64.152215, -22.002099 (Lat,Lon), so I make a point of this:

In [ ]:
# A list with a single Shapely Point object is created with (x,y)
my_house = [Point(-22.002099, 64.152215)]

Next, I load it into a geodataframe and snap it to the network:

In [ ]:
mini_gdf = gpd.GeoDataFrame(
    {"geometry": my_house}, crs="epsg:4326", geometry="geometry", index=[1]
)

In [ ]:
mini_gdf

In [ ]:
origin_gdf = gn.pandana_snap_c(
    G_time, mini_gdf, source_crs="epsg:4326", target_crs="epsg:32627"
)

In [ ]:
origin_gdf

In [ ]:
# This is the nearest node (NN)
origin_gdf.iloc[0].NN

Now, We can calculate the OD matrix using the GOSTNets calculate_OD function. Bear in mind it takes list objects as inputs:

In [ ]:
origin = [origin_gdf.iloc[0].NN]

In [ ]:
OD = gn.calculate_OD(G_time, origin, destinations, fail_value=9999999)

The OD matrix displays the time in seconds to reach each church

In [ ]:
OD

We can use minutes as the measure by dividing every value in the OD Matrix by 60. Then we can convert the array nicely into a pandas Dataframe, 

In [ ]:
OD = OD / 60
OD_df = pd.DataFrame(OD, columns=destinations, index=origin)

In [ ]:
OD_df

It appears nearly all of the churches less than twenty minutes away.

Now that you are up to speed on the basics, check out some of the example notebooks in the GOSTNets_PublicGoods 'Implementations' folder.